In [2]:
#!/usr/bin/env python
# coding: utf-8

# This script is used to compare two ensemble outputs (e.g., gauge-based GMET and NLDAS-based GMET)

from mpl_toolkits.basemap import Basemap
from pyproj import Proj
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import pandas as pd
import xarray as xr
import datetime

def read_ens(out_forc_name_base, ens_num):
    for i in range(ens_num):
        ens_file = os.path.join(out_forc_name_base + '.' + str('%03d' % (i+1)) +'.nc')
        
        f=xr.open_dataset(ens_file)
        pcp = f.variables['pcp'][:]
        t_mean = f.variables['t_mean'][:]
        t_range = f.variables['t_range'][:]

        if i == 0:
            lats = f['latitude'].values[:] #shape (y,x)
            lons = f['longitude'].values[:]
            time = pd.DatetimeIndex(f['time'][:].dt.floor('D').to_pandas())
                 
            pcp_ens = np.zeros((np.shape(pcp)[0], np.shape(pcp)[1], np.shape(pcp)[2], ens_num))# create ens array 
            t_mean_ens = np.zeros((np.shape(pcp)[0], np.shape(pcp)[1], np.shape(pcp)[2], ens_num))
            t_range_ens = np.zeros((np.shape(pcp)[0], np.shape(pcp)[1], np.shape(pcp)[2], ens_num))

        pcp_ens[:,:,:,i] = pcp
        t_mean_ens[:,:,:,i] = t_mean
        t_range_ens[:,:,:,i] = t_range
       
    return lats, lons, time, pcp_ens, t_mean_ens, t_range_ens

def plot_basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,ax,lat_0,lon_0,ny,nx):

    m = Basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,resolution='l',projection='cyl', ax=ax)   
#     m = Basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,resolution='l',projection='tmerc', ax=ax,lat_0=lat_0,lon_0=lon_0)

    m.drawstates(linewidth=1, linestyle='solid', color='grey')
    m.drawcountries(linewidth=1, linestyle='solid', color='k')
    m.drawcoastlines(linewidth=.75, linestyle='solid', color='k')
    return m

# set the colormap and centre the colorbar
class MidpointNormalize(mpl.colors.Normalize):
    """Normalise the colorbar.
    source: http://chris35wills.github.io/matplotlib_diverging_colorbar/
    e.g. im=ax1.imshow(array, norm=MidpointNormalize(midpoint=0.,vmin=-300, vmax=1000))    
    """
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        mpl.colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y), np.isnan(value))

#======================================================================================================
# main script
root_dir = '/glade/u/home/hongli/work/2020_04_21nldas_gmet'   
nldas_dir = os.path.join(root_dir,'data/nldas_daily_utc')
start_yr = 2015
end_yr = 2016

gridinfo_file = os.path.join(root_dir,'scripts/conus_ens_grid_eighth_deg_v1p1.nc')
stn_ens_dir = os.path.join(root_dir,'data/stn_ens')

nldas_ens_dir = os.path.join(root_dir,'test_uniform')
test_folders = [d for d in os.listdir(nldas_ens_dir)]
test_folders = sorted(test_folders)

ens_num = 100
time_format = '%Y-%m-%d'

dpi_value = 150
plot_date_start = '2015-01-01'
plot_date_end = '2016-12-31'
plot_date_start_obj = datetime.datetime.strptime(plot_date_start, time_format)
plot_date_end_obj = datetime.datetime.strptime(plot_date_end, time_format)

output_dir=os.path.join(root_dir, 'scripts/step9_plot_spatial_stn_NLDAS')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
of_namebase = 'spatial_'
    
#======================================================================================================
print('Read gridinfo mask')
# get xy mask from gridinfo.nc
f_gridinfo = xr.open_dataset(gridinfo_file)
mask_xy = f_gridinfo['mask'].values[:] # (y, x). 1 is valid. 0 is invalid.
data_mask = f_gridinfo['data_mask'].values[:] # (y, x). 1 is valid. 0 is invalid.

#======================================================================================================
print('Read stn ensemble')


#======================================================================================================
print('Plot')
# loop through all uniform tests
for test_folder in test_folders[0:1]:
    
    print(test_folder)
    test_dir = os.path.join(result_dir, test_folder)
    fig_title= test_folder

    # read output ensemble
    output_namebase = os.path.join(test_dir,'outputs', 'ens_forc')
    ens_lats, ens_lons, ens_time, pcp_ens, tmean_ens, trange_ens = read_ens(output_namebase, ens_num)

    # define plot mask for nldas ensemble
    mask_ens_t = (ens_time>=plot_date_start_obj) & (ens_time<=plot_date_end_obj)
    
    # calculate time-series mean values of mean and std
    pcp_ens_mean = np.nanmean(np.nanmean(pcp_ens[mask_ens_t,:,:,:], axis = 3),axis=0) 
    pcp_ens_std = np.nanmean(np.std(pcp_ens[mask_ens_t,:,:,:], axis = 3),axis=0)
    
    tmean_ens_mean = np.nanmean(np.nanmean(tmean_ens[mask_ens_t,:,:,:], axis = 3),axis=0)
    tmean_ens_std = np.nanmean(np.std(tmean_ens[mask_ens_t,:,:,:], axis = 3),axis=0)

    trange_ens_mean = np.nanmean(np.nanmean(trange_ens[mask_ens_t,:,:,:], axis = 3),axis=0)
    trange_ens_std = np.nanmean(np.std(trange_ens[mask_ens_t,:,:,:], axis = 3),axis=0)
    
    # convert masked values to nan
    pcp_ens_mean=np.where(mask_xy==0,np.nan,pcp_ens_mean)
    pcp_ens_std=np.where(mask_xy==0,np.nan,pcp_ens_std)
    
    tmean_ens_mean=np.where(mask_xy==0,np.nan,tmean_ens_mean)
    tmean_ens_std=np.where(mask_xy==0,np.nan,tmean_ens_std)
    
    trange_ens_mean=np.where(mask_xy==0,np.nan,trange_ens_mean)
    trange_ens_std=np.where(mask_xy==0,np.nan,trange_ens_std)
    
    # setup plot colorbar range for the plot_date
    vmin_prcp_mean=np.nanmin([np.nanmin(prcp_mean), np.nanmin(pcp_ens_mean)])
    vmax_prcp_mean=np.nanmax([np.nanmax(prcp_mean), np.nanmax(pcp_ens_mean)])
    vmin_prcp_std=np.nanmin(pcp_ens_std)
    vmax_prcp_std=np.nanmax(pcp_ens_std)
    
    vmin_t_mean_mean=np.nanmin([np.nanmin(tmean_mean), np.nanmin(tmean_ens_mean)])
    vmax_t_mean_mean=np.nanmax([np.nanmax(tmean_mean), np.nanmax(tmean_ens_mean)])
    vmin_t_mean_std=np.nanmin(tmean_ens_std)
    vmax_t_mean_std=np.nanmax(tmean_ens_std)
    
    vmin_t_range_mean=np.nanmin([np.nanmin(trange_mean), np.nanmin(trange_ens_mean)])
    vmax_t_range_mean=np.nanmax([np.nanmax(trange_mean), np.nanmax(trange_ens_mean)])
    vmin_t_range_std=np.nanmin(trange_ens_std)
    vmax_t_range_std=np.nanmax(trange_ens_std)

    # plot
    nrow = 3 # prcp, tmean, trange
    ncol = 3 # NLDAS, ens mean, ens std
    fig, ax = plt.subplots(nrow, ncol)#, constrained_layout=True)
    fig.set_figwidth(3.54*ncol) 
    fig.set_figheight(3.54*0.75*nrow)
#     fig.set_figwidth(7.08) 
#     fig.set_figheight(7.08*0.75)
    fig.suptitle(fig_title, fontsize='medium', fontweight='semibold', color='g')

    llcrnrlon = ens_lons[0,0]
    urcrnrlon = ens_lons[-1,-1]
    llcrnrlat = ens_lats[0,0]
    urcrnrlat = ens_lats[-1,-1]
    lat_0=0.5*(llcrnrlat+urcrnrlat)
    lon_0=0.5*(llcrnrlon+urcrnrlon)
    (ny,nx)=np.shape(ens_lats)
    
    for i in range(nrow):
        for j in range(ncol):
    
            # plot Basemap
            m = plot_basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,ax[i,j],lat_0,lon_0,ny,nx) # plot Basemap 
            
            # select data for each subplot
            # PCP (first row)
            if i == 0 and j == 0:
                data=prcp_mean
                cmap=plt.cm.Blues
                vmin=vmin_prcp_mean
                vmax=vmax_prcp_mean
                title_str = '(a) NLDAS Daily Precip'
            elif i == 0 and j == 1:
                data=pcp_ens_mean
                cmap=plt.cm.Blues
                vmin=vmin_prcp_mean
                vmax=vmax_prcp_mean
                title_str = '(b) Ens Mean of Daily Precip'
            elif i == 0 and j == 2:
                data=pcp_ens_std
                cmap=plt.cm.Blues
                vmin=vmin_prcp_std
                vmax=vmax_prcp_std
                title_str = '(c) Ens Std of Daily Precip'
    
            # T_MEAN (second row)
            elif i == 1 and j == 0:
                data=tmean_mean
                cmap=plt.cm.Reds
                vmin=vmin_t_mean_mean
                vmax=vmax_t_mean_mean
                title_str = '(d) NLDAS Mean Temp'
            elif i == 1 and j == 1:
                data=tmean_ens_mean
                cmap=plt.cm.Reds
                vmin=vmin_t_mean_mean
                vmax=vmax_t_mean_mean
                title_str = '(e) Ens Mean of Mean Temp'
            elif i == 1 and j == 2:
                data=tmean_ens_std
                cmap=plt.cm.Reds
                vmin=vmin_t_mean_std
                vmax=vmax_t_mean_std
                title_str = '(f) Ens Std of Mean Temp'
    
            # T_RANGE (third row)
            elif i == 2 and j == 0:
                data=trange_mean
                cmap=plt.cm.Greens
                vmin=vmin_t_range_mean
                vmax=vmax_t_range_mean
                title_str = '(g) NLDAS Temp Range'
            elif i == 2 and j == 1:
                data=trange_ens_mean
                cmap=plt.cm.Greens
                vmin=vmin_t_range_mean
                vmax=vmax_t_range_mean
                title_str = '(h) Ens Mean of Temp Range'
            elif i == 2 and j == 2:
                data=trange_ens_std
                cmap=plt.cm.Greens #bwr
                vmin=vmin_t_range_std
                vmax=vmax_t_range_std
                title_str = '(i) Ens Std of Temp Range'

            # plot data
            im1 = m.pcolormesh(ens_lons, ens_lats,data,shading='flat',latlon=True,cmap=cmap,vmin=vmin,vmax=vmax)

            # set title
            ax[i,j].set_title(title_str, fontsize='small', fontweight='semibold')

            # set colorbar
            cbar = m.colorbar(im1, location='right')
            if i == 0:
                cbar.set_label(label='(mm/day)', size='small', rotation='horizontal', labelpad=-20, y=1.1) #y=1.04
            elif i >= 1:
                cbar.set_label(label='($^\circ$C)', size='small', rotation='horizontal', labelpad=-20, y=1.1)
            cbar.ax.tick_params(labelsize='small') 
    
    # save plot
    fig.tight_layout(pad=0.1, w_pad=0.05, h_pad=0.00)
    output_filename = of_namebase+test_folder+'.png'
    fig.savefig(os.path.join(output_dir, output_filename), dpi=dpi_value)
    plt.close(fig)
    #     plt.show()
    del pcp_ens, tmean_ens, trange_ens

print('Done')


Read gridinfo mask
Read nldas data


/glade/u/home/hongli/tools/miniconda/ENTER/lib/python3.7/site-packages/ipykernel_launcher.py:129: RuntimeWarning: Mean of empty slice
/glade/u/home/hongli/tools/miniconda/ENTER/lib/python3.7/site-packages/ipykernel_launcher.py:130: RuntimeWarning: Mean of empty slice
/glade/u/home/hongli/tools/miniconda/ENTER/lib/python3.7/site-packages/ipykernel_launcher.py:131: RuntimeWarning: Mean of empty slice
/glade/u/home/hongli/tools/miniconda/ENTER/lib/python3.7/site-packages/xarray/core/dataarray.py:1658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  3: pd.Panel}


Plot
00822grids_test


/glade/u/home/hongli/tools/miniconda/ENTER/lib/python3.7/site-packages/ipykernel_launcher.py:42: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.
/glade/u/home/hongli/tools/miniconda/ENTER/lib/python3.7/site-packages/ipykernel_launcher.py:45: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.
/glade/u/home/hongli/tools/miniconda/ENTER/lib/python3.7/site-packages/ipykernel_launcher.py:46: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.


Done


In [18]:
np.shape(prcp_mean)

(array([], dtype=int64),)